<center><h1>TOÁN ỨNG DỤNG VÀ THỐNG KÊ</h1>
<h2>Đồ án Diagonalizable matrix</h2>
<b>Họ và tên:</b> Nguyễn Trần Thiên Phú <br>
<b>MSSV:</b> 23127246</center>

## 1. Giải thuật

### 1. Thuật toán Gauss - Jordan
**Gauss - Jordan** là một thuật toán để tìm ma trận ma trận nghịch đảo của một ma trận thông qua các phép biến đổi sơ cấp trên dòng. 

> **Bước 1:** Tạo ma trận bổ sung $(A|I)$ với A là ma trận cần nghịch đảo, I là ma trận đơn vị

> **Bước 2:** Áp dụng các phép biến đổi sơ cấp trên dòng để sao cho biển đổi ma trận bên trái - ma trận A - thành ma trận đơn vị

> **Bước 3:** Thu được ma trận có dạng $(I|A^{-1})$ nếu ma trận A khả nghịch, còn nếu không thu được ma trận I ở bên trái thì ma trận A không khả nghịch


### 2. Chéo hóa ma trận (Diagonalize)

> **Bước 1:** Tính $A- \lambda I_n$ sau đó giải phương trình det($A- \lambda I_n$) = 0 để xác định các trị riêng $\lambda$

> **Bước 2:** Với mỗi $\lambda$, tìm cơ sở v cho không gian riêng ($A - \lambda I_n$)v = 0

> **Bước 3:** Xác định nếu số vector riêng bằng n thì ma trận chéo hóa được, không thì ma trận không chéo hóa được, bài toán kết thúc

> **Bước 4:** Dựng ma trận P bằng cách dựng các cơ sở riêng v thành các cột

> **Bước 5:** Tính ma trận D với $D = P^{-1}.A.P$. Kết thúc

In [46]:
def print_matrix(matrix):
    matrix = [[round(elem, 2) for elem in row] for row in matrix]

    str_matrix = [[str(item) for item in row] for row in matrix]

    num_cols = len(str_matrix[0])
    col_widths = [0] * num_cols
    for j in range(num_cols):
        max_len = max(len(row[j]) for row in str_matrix)
        col_widths[j] = max_len

    # In ma trận đã được định dạng
    num_rows = len(str_matrix)
    for i, row in enumerate(str_matrix):
        line_str = "  "
        
        if i == 0:
            line_str += '┌'
        elif i == num_rows - 1:
            line_str += '└'
        else:
            line_str += '│'
        
        line_str += ' '
        
        for j, element in enumerate(row):
            line_str += element.rjust(col_widths[j]) + '  '
            
        line_str = line_str.rstrip()
        line_str += ' '

        if i == 0:
            line_str += '┐'
        elif i == num_rows - 1:
            line_str += '┘'
        else:
            line_str += '│'
        
        print(line_str)


### Tìm ma trận nghịch đảo bằng thuật toán Gauss-Jordan

In [47]:
def Gauss_Jordan(matrix):
  
    n = len(matrix)
    if any(len(row) != n for row in matrix):
        raise ValueError("Lỗi: Ma trận đầu vào phải là ma trận vuông.")

    # Tạo một bản sao của ma trận gốc để không thay đổi nó
    A = [row[:] for row in matrix]
    
    # Tạo ma trận bổ sung (A|I)
    # Ma trận I được ghép vào cuối mỗi hàng của A
    for i in range(n):
        identity_row = [1.0 if j == i else 0.0 for j in range(n)]
        A[i].extend(identity_row)

    # Áp dụng thuật toán Gauss-Jordan để biến đổi A thành ma trận đơn vị
    for i in range(n):
        # Tìm hàng có pivot lớn nhất và đổi chỗ (Partial Pivoting)
        max_row = i
        for k in range(i + 1, n):
            if abs(A[k][i]) > abs(A[max_row][i]):
                max_row = k
        A[i], A[max_row] = A[max_row], A[i]

        # Kiểm tra ma trận khả nghịch
        pivot = A[i][i]
        if abs(pivot) < 1e-10: # Sử dụng một ngưỡng nhỏ để xử lý sai số dấu phẩy động
            raise ValueError("Lỗi: Ma trận không khả nghịch (singular matrix).")

        # Chuẩn hóa hàng pivot (biến pivot thành 1)
        for j in range(i, 2 * n):
            A[i][j] /= pivot

        # 2d. Khử các phần tử khác trong cột pivot (biến chúng thành 0)
        for k in range(n):
            if k != i:
                factor = A[k][i]
                for j in range(i, 2 * n):
                    A[k][j] -= factor * A[i][j]

    # Trích xuất ma trận nghịch đảo từ phần bên phải của ma trận bổ sung
    inverse_matrix = [row[n:] for row in A]

    return inverse_matrix

### Thuận toán xác định trị riêng của ma trận

In [48]:
def quadratic_slt(a, b, c):
    if a == 0:
        if b == 0:
            if c != 0:
                raise ValueError("Phương trình vô nghiệm.")
            else:
                return ValueError("Phương trình vô số nghiệm.")
        else:
            return [-c / b]
    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        raise ValueError("Phương trình vô nghiệm (discriminant < 0).")
    elif discriminant == 0:
        return [-b / (2*a)]
    else:
        sqrt_discriminant = discriminant**0.5
        return [(-b + sqrt_discriminant) / (2*a), (-b - sqrt_discriminant) / (2*a)]

PI = 3.141592653589793
TOLERANCE = 1e-15  # Sai số chấp nhận được để dừng vòng lặp

def my_cos(x):
    x = x % (2 * PI)
    if x > PI: x -= 2 * PI
    if x < -PI: x += 2 * PI
    term = 1.0; total_sum = term; n = 1
    while abs(term) > TOLERANCE:
        term = -term * x**2 / ((2 * n - 1) * (2 * n))
        total_sum += term
        n += 1
    return total_sum

def my_arccos(x):
    if not -1 <= x <= 1: return float('nan')
    if x == 1.0: return 0.0
    if x == -1.0: return PI
    x_squared = x**2; term = x; arcsin_sum = term; n = 1
    while abs(term) > TOLERANCE:
        numerator = (2 * n - 1) ** 2
        denominator = (2 * n) * (2 * n + 1)
        term = term * x_squared * numerator / denominator
        arcsin_sum += term
        n += 1
    return PI / 2 - arcsin_sum

def cbrt(x):
    if x >= 0:
        return x**(1/3)
    else:
        return -(-x)**(1/3)

def cubic_slt(a, b, c, d, precision=10):
    if a == 0:
        raise ValueError("Hệ số 'a' không thể bằng 0 cho phương trình bậc 3.")

    # 1. Đưa về dạng t³ + pt + q = 0
    p = (3*a*c - b**2) / (3*a**2)
    q = (2*b**3 - 9*a*b*c + 27*a**2*d) / (27*a**3)
    
    # 2. Tính biệt thức Delta
    delta = (q/2)**2 + (p/3)**3

    offset = -b / (3*a)
    roots = []

    # 3. Xét các trường hợp dựa trên Delta
    if delta > 0:
        # Trường hợp có 1 nghiệm thực
        sqrt_delta = delta**0.5
        u = cbrt(-q/2 + sqrt_delta)
        v = cbrt(-q/2 - sqrt_delta)
        t1 = u + v
        roots = [t1 + offset]

    elif delta == 0:
        # Trường hợp có nghiệm bội (2 hoặc 3 nghiệm bằng nhau)
        # SỬA LỖI LOGIC TẠI ĐÂY
        t1 = 2 * cbrt(-q/2)
        t2 = -cbrt(-q/2) # Nghiệm kép
        roots = sorted([
            t1 + offset,
            t2 + offset,
            t2 + offset # Thêm nghiệm kép một lần nữa
        ])

    else: # delta < 0: Trường hợp 3 nghiệm thực phân biệt
        phi_arg = (3 * q) / (2 * p) * (-3 / p)**0.5
        phi = my_arccos(phi_arg)
        t_factor = 2 * (-p/3)**0.5
        
        t0 = t_factor * my_cos(phi / 3)
        t1 = t_factor * my_cos(phi / 3 - 2 * PI / 3)
        t2 = t_factor * my_cos(phi / 3 + 2 * PI / 3)
        
        roots = sorted([
            t0 + offset,
            t1 + offset,
            t2 + offset
        ])
    
    # Làm tròn tất cả các nghiệm tìm được trước khi trả về
    return [round(r, precision) for r in roots]

def trace(A):
    return sum(A[i][i] for i in range(len(A)))

def determinant_3x3(A):
    a, b, c = A[0]
    d, e, f = A[1]
    g, h, i = A[2]
    return a*(e*i - f*h) - b*(d*i - f*g) + c*(d*h - e*g)

def multi_matrix(A, B):
    n = len(A)
    C = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    return C

import numpy as np

def Find_Eigenvalues(A):
    # 1. Kiểm tra đầu vào
    if not isinstance(A, list) or not all(isinstance(row, list) for row in A):
        raise TypeError("Đầu vào phải là một list các list (ma trận).")
    n = len(A)
    if any(len(row) != n for row in A):
        raise ValueError("Ma trận phải là ma trận vuông.")

    eigenvalues = []

    # 2. Sử dụng các thuật toán nhỏ để xác định hệ số phương trình đặc trưng
    # Phương trình đặc trưng: det(A - λI) = 0

    if n == 1:
        eigenvalues = [A[0][0]]
    elif n == 2:
        # λ² - trace(A)λ + det(A) = 0
        tr_A = A[0][0] + A[1][1]
        det_A = A[0][0]*A[1][1] - A[0][1]*A[1][0]
        eigenvalues = quadratic_slt(1, -tr_A, det_A)
    elif n == 3:
        # λ³ - trace(A)λ² + 0.5*(trace(A)² - trace(A²))λ - det(A) = 0
        tr_A = trace(A)
        det_A = determinant_3x3(A)
        A_squared = multi_matrix(A, A)
        tr_A2 = trace(A_squared)
        
        # Hệ số của phương trình bậc 3
        a_poly = 1
        b_poly = -tr_A
        c_poly = 0.5 * (tr_A**2 - tr_A2)
        d_poly = -det_A
        
        eigenvalues = cubic_slt(a_poly, b_poly, c_poly, d_poly)
    else: # n >= 4
        # 3. Với bậc 4 trở lên, sử dụng thư viện như mô tả
        print(f"Ma trận bậc {n} >=4, sử dụng thư viện NumPy để tính toán.")
        eigenvalues = list(np.linalg.eigvals(A))

    # 4. Trả về mảng chứa trị riêng tăng dần và ma trận đường chéo D
    eigenvalues.sort()
    D = [[0] * n for _ in range(n)]
    for i in range(n):
        D[i][i] = eigenvalues[i]
        
    return eigenvalues, D

### Thuật toán xác định các vector riêng của ma trận

In [49]:
def sub_matrix(A,B):
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        raise ValueError("Ma trận A và B phải có cùng kích thước.")
    
    return [[A[i][j] - B[i][j] for j in range(len(A[0]))] for i in range(len(A))]

def scalar_multi_matrix(A,n):
    return [[elem * n for elem in row] for row in A]

def transpose_matrix(A):
    return [[A[j][i] for j in range(len(A))] for i in range(len(A[0]))]

def homogeneous_system_slt(B):
    M = [row[:] for row in B] # Tạo bản sao
    n = len(M)
    
    # 1. Phép khử Gauss
    for i in range(n):
        pivot_row = i
        while pivot_row < n and abs(M[pivot_row][i]) < 1e-9:
            pivot_row += 1
        if pivot_row == n: continue
        M[i], M[pivot_row] = M[pivot_row], M[i]
        pivot_val = M[i][i]
        M[i] = [x / pivot_val for x in M[i]]
        for j in range(n):
            if i != j:
                factor = M[j][i]
                M[j] = [M[j][k] - factor * M[i][k] for k in range(n)]

    # 2. Tìm nghiệm từ ma trận đã khử
    v = [0] * n
    v[n-1] = 1 # Giả sử biến cuối cùng là biến tự do và đặt bằng 1
    for i in range(n - 2, -1, -1):
        # Tìm các hàng không phải là hàng không
        pivot_found = False
        for j in range(n):
            if abs(M[i][j] - 1) < 1e-9: # Tìm pivot
                sum_val = sum(M[i][k] * v[k] for k in range(j + 1, n))
                v[j] = -sum_val
                pivot_found = True
                break
        if not pivot_found: v[i] = 1 # Nếu cột không có pivot -> biến tự do
            
    # Chuẩn hóa vector
    norm = sum(x**2 for x in v)**0.5
    return [x / norm for x in v] if norm != 0 else v

def identity_matrix(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]
    
def Find_Eigenvectors(A, eigenvalues):
    n = len(A)
    # 1. Đầu vào là mảng các trị riêng (và ma trận A)
    I = identity_matrix(n)
    eigenvectors = []

    # Xử lý các trị riêng duy nhất để tránh tính toán thừa
    unique_eigenvalues = sorted(list(set(eigenvalues)))

    for val in unique_eigenvalues:
        # 2. Gọi hàm tính toán ma trận để tìm cơ sở v
        #    Tính B = A - λI
        lambda_I = scalar_multi_matrix(I, val)
        B = sub_matrix(A, lambda_I)
        
        # Tìm một vector v trong không gian rỗng của B
        v = homogeneous_system_slt(B)
        eigenvectors.append(v)

    # 3. Kiểm tra số lượng vector riêng tìm được
    if len(eigenvectors) < n:
        return None

    # 4. Nếu chéo hóa được, chuyển các vector riêng thành ma trận P
    #    Các vector riêng hiện là các hàng, cần chuyển vị để thành các cột
    P = transpose_matrix(eigenvectors)
    
    return P
    


### Chéo hóa ma trận

In [50]:
def Diagonalize_Matrix(A):
    print("="*60)
    print(f"Bắt đầu chéo hóa ma trận A:")
    print_matrix(A)
    print("\nBước 1: Tìm trị riêng và ma trận đường chéo D:")
    eigenvalues, D = Find_Eigenvalues(A)
    if not eigenvalues:
        print("Không tìm thấy trị riêng thực.")
        return
    print(f"-> Ma trận đường chéo D:")
    print_matrix(D)
    
    print("\nBước 2: Tìm các vector riêng và ma trận P:")
    P = Find_Eigenvectors(A, eigenvalues) 
    if not P:
        print("Không tìm thấy đủ vector riêng độc lập để chéo hóa ma trận.")
        return 
    print(f"-> Ma trận P:")
    print_matrix(P)
    print(f"-> Ma trận P⁻¹:")
    print_matrix(Gauss_Jordan(P))
        
        

A = [[4, 0, -1], 
     [0, 3, 0],
     [1, 0, 2]]
Diagonalize_Matrix(A)


Bắt đầu chéo hóa ma trận A:
  ┌ 4  0  -1 ┐
  │ 0  3   0 │
  └ 1  0   2 ┘

Bước 1: Tìm trị riêng và ma trận đường chéo D:
-> Ma trận đường chéo D:
  ┌ 3.0    0    0 ┐
  │   0  3.0    0 │
  └   0    0  3.0 ┘

Bước 2: Tìm các vector riêng và ma trận P:
Không tìm thấy đủ vector riêng độc lập để chéo hóa ma trận.


### Ý tưởng thực hiện và mô tả các hàm

#### `Gauss_Jordan(A)`
**Ý tưởng:** Hàm này mô tả lại quá trình tìm ma trận nghịch đảo của 1 ma trận bằng thuật toán Gauss_Jordan:

**Mô tả hoạt động:** 


#### `Find_Eigenvalues(A)`
**Ý tưởng:** Hàm này mô tả lại quá trình tính các trị riêng của 1 ma trận cùng các hàm hỗ trợ:
- `my_cos(x)`: Hàm tính `cos(x)` bằng khai triển Taylor
- `my_arccos(x)`: Hàm tính `arcsin(x)` sau đó tính `arccos(x)` = $\pi/2$ - `arcsin(x)`
- `cbrt(x)`: Hàm tính căn bậc 3, xử lí cả số âm
- `quadratic_slt`: Hàm giải phương trình bậc 2
- `cubic_slt`: Hàm giải phương trình bậc 3 theo công thức nghiệm 
- `trace(A)`: Hàm tính vết của ma trận vuông 
- `determinant_3x3(A)`: Hàm tính định thức của ma trận bậc 3
- `multi_matrix(A,B)`: Hàm trả về kết quả phép nhân ma trận A x B <br>
**Mô tả hoạt động:**
1. Đầu vào là 1 ma trận vuông A bậc n, nếu không phải ma trận vuông thì kết thúc thuật toán
2. Sử dụng các công thức để xác định được các hệ số cho phương trình tìm trị riêng bậc 2 và 3
3. Gọi hàm tương ứng với bậc của phương trình sau đó trả về các trị riêng với các phương trình bậc 2,3 còn bậc 4 thì sử dụng thư viện
4. Trả về mảng chứa các trị riêng tăng dần và ma trận đường chéo D gồm các trị riêng

#### `Find_Eigenvector(A)`
**Ý tưởng:** Hàm này mô tả lại quá trình tính các vector riêng từ các trị riêng vừa tìm, trả về ma trận chéo P cùng các hàm hỗ trợ:
- `sub_matrix(A,B)`: Hàm trả về kết quả phép trừ ma trận A - B
- `scalar_multi_matrix(A,n)`: Hàm trả về kết quả phép nhân của ma trận với 1 hằng số 
- `transpose_matrix(A)`: Chuyển vị ma trận, chuyển hàng thứ i thành cột thứ i
- `homogenous_system_slt(A)`: Hàm trả về cơ sở v cho không gian riêng ($A - \lambda I_n$)v = 0 <br>
**Mô trả hoạt động:**
1. Đầu vào là mảng gồm các trị riêng tăng dần
2. Gọi 2 hàm tính toán ma trận và sử dụng thuật toán Gauss để tìm cơ sở v cho không gian riêng ($A - \lambda I_n$)v = 0
3. Xác định nếu số vector riêng bằng kích thước ma trận thì chéo hóa được, không thì ma trận không chéo hóa được, bài toán kết thúc
4. Nếu chéo hóa được, chuyển các vector riêng thành 1 hàng của 1 ma trận sau đó dùng hàm chuyển vị ma trận để chuyển thành ma trận P

#### `Diagonalize_Matrix(A)`
**Ý tưởng:** Hàm tổng hợp lại quá trình chéo hóa ma trận dựa trên 2 hàm chính:
- `Find_Eigenvalues(A)`: Hàm xác định trị riêng
- `Find_Eigenvectors(A, eigenvalues)`: Hàm xác định vector riêng <br>
**Mô tả hoạt động:** 
1. Lần lượt gọi các hàm tìm trị riêng, tìm vector riêng đã mô tả
2. Xác định các ma trận $P, D, P^{-1}$ và in ra nếu ma trận chéo hóa được



## 2. Mở rộng

### Ứng dụng của chéo hóa ma trận
Chéo hóa ma trận là một công cụ cực kỳ mạnh mẽ trong toán học và các ngành khoa học ứng dụng. Việc biến đổi ma trận A thành dạng $PDP^{-1}$ mang lại nhiều lợi ích, đặc biệt là trong tính toán. Dưới đây là một số ứng dụng quan trọng:

*   **Tính lũy thừa của ma trận:** Đây là ứng dụng phổ biến nhất. Thay vì nhân ma trận A với chính nó k lần (phép toán rất tốn kém), ta có thể tính $A^k$ một cách hiệu quả:
    $A^k = (PDP^{-1})^k = PDP^{-1}PDP^{-1}...PDP^{-1} = PD^kP^{-1}$
    Việc tính $D^k$ rất đơn giản, chỉ cần lấy lũy thừa của các phần tử trên đường chéo. Ứng dụng này rất quan trọng trong việc mô tả các chuỗi Markov, hệ thống động lực học.

*   **Giải hệ phương trình vi phân tuyến tính:** Các hệ phương trình có dạng $\mathbf{x}'(t) = A\mathbf{x}(t)$ có thể được giải quyết dễ dàng bằng cách chéo hóa A. Phép biến đổi $\mathbf{y} = P^{-1}\mathbf{x}$ sẽ đưa hệ phương trình về dạng $\mathbf{y}'(t) = D\mathbf{y}(t)$, một hệ gồm các phương trình vi phân độc lập và rất dễ giải.

*   **Phân tích thành phần chính (Principal Component Analysis - PCA):** Trong thống kê và học máy, PCA là một kỹ thuật giảm chiều dữ liệu. Nó hoạt động bằng cách chéo hóa ma trận hiệp phương sai của dữ liệu. Các vector riêng (thành phần chính) tương ứng với các trị riêng lớn nhất sẽ chỉ ra các chiều có phương sai lớn nhất (chứa nhiều thông tin nhất) trong dữ liệu.

### So sánh thuật toán tự viết với thư viện (NumPy)
Để đánh giá độ chính xác và hiệu quả của thuật toán tự cài đặt, ta sẽ so sánh kết quả với hàm `numpy.linalg.eig` - một hàm được tối ưu hóa và kiểm thử rộng rãi trong thư viện NumPy.

In [51]:

print("="*60)
print("Sử dụng thư viện NumPy để chéo hóa ma trận A:")
A_np = np.array(A) # A là ma trận đã định nghĩa ở trên
print_matrix(A_np.tolist())

# NumPy trả về trị riêng và vector riêng (đã được chuẩn hóa)
eigenvalues_np, P_np = np.linalg.eig(A_np)

# Sắp xếp để dễ so sánh
sorted_indices = np.argsort(eigenvalues_np)
eigenvalues_np = eigenvalues_np[sorted_indices]
P_np = P_np[:, sorted_indices]

# Tạo ma trận đường chéo D từ trị riêng
D_np = np.diag(eigenvalues_np)

print("\n-> Các trị riêng tìm được bởi NumPy:", [round(e, 2) for e in eigenvalues_np])

print("\n-> Ma trận đường chéo D (từ NumPy):")
print_matrix(D_np.tolist())

print("\n-> Ma trận P (các vector riêng) (từ NumPy):")
print_matrix(P_np.tolist())

try:
    P_inv_np = np.linalg.inv(P_np)
    print("\n-> Ma trận P⁻¹ (từ NumPy):")
    print_matrix(P_inv_np.tolist())
except np.linalg.LinAlgError:
    print("\nNumPy không thể tính P⁻¹ vì P là ma trận suy biến (không khả nghịch).")


Sử dụng thư viện NumPy để chéo hóa ma trận A:
  ┌ 4  0  -1 ┐
  │ 0  3   0 │
  └ 1  0   2 ┘

-> Các trị riêng tìm được bởi NumPy: [np.float64(3.0), np.float64(3.0), np.float64(3.0)]

-> Ma trận đường chéo D (từ NumPy):
  ┌ 3.0  0.0  0.0 ┐
  │ 0.0  3.0  0.0 │
  └ 0.0  0.0  3.0 ┘

-> Ma trận P (các vector riêng) (từ NumPy):
  ┌ 0.71  0.71  0.0 ┐
  │  0.0   0.0  1.0 │
  └ 0.71  0.71  0.0 ┘

-> Ma trận P⁻¹ (từ NumPy):
  ┌  2251799813685248.5   0.0  -2251799813685247.0 ┐
  │ -2251799813685248.0  -0.0   2251799813685247.5 │
  └                 0.0   1.0                  0.0 ┘


#### Phân tích kết quả:

*   **Về Trị riêng:** Cả thuật toán tự viết và thư viện NumPy đều tìm ra cùng một tập hợp các trị riêng cho ma trận A là `{3, 3, 3}`. Điều này cho thấy hàm `Find_Eigenvalues` hoạt động chính xác cho trường hợp ma trận 3x3 này.

*   **Về Vector riêng và Khả năng chéo hóa:**
    *   **Thuật toán tự viết:** Đưa ra kết luận "Không tìm thấy đủ vector riêng độc lập". Điều này xảy ra vì ma trận A có trị riêng `λ = 3` với bội đại số là 3. Để chéo hóa được, không gian riêng ứng với `λ = 3` cũng phải có chiều là 3 (bội hình học = 3).
    *   **Thư viện NumPy:** Vẫn tìm được một ma trận P gồm các vector riêng. Tuy nhiên, ma trận P này có 2 cột giống nhau nên chỉ đưa ra được 2 vector riêng cho ma trận A.

*   **Kết luận so sánh:**
    Cả hai đều giải được bài toán tuy nhiên thuật toán tự viết sẽ cho thấy ma trận A không chéo hóa được vì không đủ các vector riêng độc lập còn thư viện sẽ trả về ma trận P với 2 cột giống nhau